# Data Leakage from Gradients

*文章已经放在资源文件中*，文件位置：resources\literature\federated learning\fl-1.pdf

## Abstract

1.	人们认为共享梯度是安全的，事实上不是。
2.  本文实验了利用共享梯度可以在 NLP 领域可以获得 token 级别的原始数据，在 CV 领域可以获得像素级别的数据。
3.  讨论了几种避免这种数据泄露的策略。其中，在不改变训练设定的情况下，最有效的办法是梯度截断。
4.  提高人们的安全意识。


## Introduction

### 分布式训练：

- 在大规模数据集上可以加速训练过程。
- 计算在各个 worker 上执行，通过交换梯度进行同步。
- 数据是分散的，数据属于各个 worker，不同的 worker 只有在训练的过程中交换梯度。
- 可以利用不同来源的数据共同训练一个模型。

如果数据需要隐私保护的，这种模式叫做**协同训练**。

### 共享梯度真的安全么？

* 直觉：
    - 共享梯度不会泄露原始数据。
    - 给定学习模型 $F$ 、模型参数 $W$、共享梯度 $\nabla w$ 和标签对，是**无法**反向得到训练数据的。
* 实际：
    - 可以利用共享梯度判断具有某些属性的数据是否存在于一批数据中。
    - 通过 GAN 可以生成类似于接近于训练数据的数据。
    - 给定学习模型 $F$ 、模型参数 $W$、共享梯度 $\nabla w$ 和标签对，是**可以**反向得到训练数据的。
* 发生数据泄露的地方：
    - 存在中心节点的环境下，中心节点可以通过梯度获取原始训练数据。
    - 无中心节点的情况下，下一个节点可以获取上一个节点的原始训练数据。
    - 如图所示：

![data-leakage-location](https://github.com/mayday-975/Machine-Learning-Literature-Review/blob/main/resources/images/federated%20learning/fl-1/data-leakage-location.png?raw=true)

### 文章的贡献

* 提出并实现了 DLG 算法
    - 快速，仅需要几次迭代，就可以通过公开的梯度获取原始训练数据和其标签对。
    - 精确，可以获取像素级别或者 token 级别的训练数据。之前的方法只是近似的结果。
    - 依赖少：
        - 无需对训练数据做出假设
        - 不依赖任何生成模型   
* 分析了不同场景下攻击的难点。
* 讨论了面对攻击的几种不同防卫策略：
    - 梯度扰动：加入高斯和拉普拉斯噪声
    - 低精度：精度降低一半
    - 梯度压缩：梯度截断 20%

## Related Work

* 分布式训练
    - 背景：海量数据并且计算密集，单机训练很慢
    - 策略：从算法和框架上并行计算，加速计算
        - 计算框架：
            - 中心化：Parameter Server
            - 非中心化：P2P 梯度交换
        - 更新算法：同步 SGD，原因性能稳定
* 协同训练：
    - 考虑数据隐私的分布式训练
* “Shallow” Leakage from Gradients
    - 核心思想：梯度可能泄露数据
    - 例子：
        - NLP
            - embedding 层可能泄露哪些词存在，但无法给定词之间的顺序。
            - ！！！~~观察全连接层的梯度变化可以推测出输出特征的值~~(不知道原文中讲的计算任务是啥，也没给出相关文献，所以忽略)
        - CV 任务中，可以利用 GAN 生成与训练数据相似的图片，仅仅是相似，并非原始数据。
        

## Method

在标准的分布式同步训练中，每一个迭代 $t$，每一个节点 $i$，都会从其数据集中采样一批数据并计算出梯度：

$$
    \nabla W_{t,i} = \frac{\partial l(F(X_{t, i}, W_t), y_{t, i}) }{\partial W_t}
$$

$N$ 个机器上计算出的平均梯度：

$$
    \overline{\nabla W_t} = \frac{1}{N} \sum_{j}^{N} \nabla W_{t, j} \quad W_{t+1} = W_t - \eta \overline{\nabla W_t}
$$

如果想从第 $k$ 个参与者里面偷数据，那么在具有 Parameter Server 的环境中，中心节点上可以得到第 $k$ 个参与者的 $\nabla W_{t,k}$，利用这个 $\nabla W_{t,k}$ 和 DLG 算法可以恢复出原始的训练数据。

### DLG 算法：

![dlg-algorithm](https://github.com/mayday-975/Machine-Learning-Literature-Review/blob/main/resources/images/federated%20learning/fl-1/dlg-algorithm.png?raw=true)

翻译一下上面的算法：

1. 随机生成输入数据 $x'_1$ 和其对应的标签 $y'_1$
2. 通过真实的模型计算出 $x'_1$ 和 $y'_1$ 上的梯度 $\nabla W'_i$
3. 比较 $\nabla W'_i$ 和真实的 $\nabla W$ 的区别，使用平方损失 $\mathbb{D}_i = ||\nabla W'_i - \nabla W||^2$
4. 更新 $x'_i \leftarrow x'_i - \eta \nabla_{x'_i} \mathbb{D}_i$，更新 $y'_i \leftarrow y'_i - \eta \nabla_{y'_i} \mathbb{D}_i$
5. 步骤3、步骤4 重复几轮过后就能得到真实原始图片了。

用图示意一下：

![dlg-process](https://github.com/mayday-975/Machine-Learning-Literature-Review/blob/main/resources/images/federated%20learning/fl-1/dlg-process.png?raw=true)

## Experiments

### 图像分类上的泄露

效果如下图

![dlg-effect](https://github.com/mayday-975/Machine-Learning-Literature-Review/blob/main/resources/images/federated%20learning/fl-1/deep-leakage-on-image-classification.png?raw=true)

## Defense Strategies

1. 在梯度上增加噪声，噪声的量级要大于 $10^{-2}$，才会有效果。
2. 降低梯度精度，如 float16 变为 int8。
3. 梯度截断，将梯度从小到大排列，前 20% 的梯度截断为 0。  